In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

In [3]:
%%px
# Import the libraries

import numpy as np
import pyfftw as ft 
from mpi4py import MPI
import math
import sys
import os
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

from fft3d import FFT3Dfield_new
from EnergySpectrum import EnergySpectrum

In [4]:
%%px

Nx = isotropic1024coarse['nx']
Ny = isotropic1024coarse['ny']
Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']
Ly = isotropic1024coarse['ly']
Lz = isotropic1024coarse['lz']
nu = isotropic1024coarse['nu']

In [5]:
%%px

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

[stdout:0] 
n_proc = 8
rank = 0


In [6]:
%%px 
# Computational Domain

nx=Nx//nproc
ny=Ny
nz=Nz
nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0#1.024# 0.0

chkSz = 32
slabs = nx//chkSz

In [7]:
%%px

vx = ft.zeros_aligned((nx,ny,nz), dtype='float32')
vy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
vz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

# Populate velocity field from the Database

if(rank==0):
    print("Starting the loading process")

##########################################
load_from_file = True

folder = "/home/idies/workspace/scratch"
filename = "kun-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

if(load_from_file):
    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    vx = content['vx']
    vy = content['vy']
    vz = content['vz']
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))
else:
    comm.Barrier(); t1=MPI.Wtime()
    lJHTDB = libJHTDB()
    lJHTDB.initialize()
    ud = []
    for k in range(slabs):
        if(rank==0):
            print(k)
        start = np.array([rank*nx+k*chkSz, 0, 0],dtype=np.int)
        width = np.array([chkSz,ny,nz],dtype=np.int)
        #start = np.array([ 0, 0, rank*nx+k*chkSz],dtype=np.int)
        ud.append(lJHTDB.getRawData(time,start,width, 
                                    data_set = 'isotropic1024coarse',
                                    getFunction = 'Velocity') )
    
    lJHTDB.finalize()
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load field from database: {0:.2f} seconds\n'.format(t2-t1))
    
    u = np.concatenate(ud,axis=2)
    comm.Barrier(); t1=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Concatenate: {0:.2f} seconds\n'.format(t1-t2))
    
    rsh = np.transpose(u,(2,1,0,3))
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Transpose: {0:.2f} seconds\n'.format(t2-t1))
    ##########################################
    
    vx[:,:,:] = rsh[:,:,:,0]
    vy[:,:,:] = rsh[:,:,:,1]
    vz[:,:,:] = rsh[:,:,:,2]
    comm.Barrier(); t1=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Splitting: {0:.2f} seconds\n'.format(t2-t1))
    
    np.savez(file,vx=vx,vy=vy,vz=vz,nproc=nproc)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))


if(rank==0):
    print("vx shape = "+str(vx.shape))       

[stdout:0] 
Starting the loading process
Finished loading
Load from disk: 110.81 seconds
vx shape = (128, 1024, 1024)


In [8]:
%%px

K = 0.5*(vx**2+vy**2+vz**2)

In [9]:
%%px

slabK = np.sum(K)

avgK=np.zeros(1,dtype='float32')

comm.Reduce([slabK,MPI.REAL],[avgK,MPI.REAL],op=MPI.SUM)
avgK = avgK[0]*(1024**(-3))
avgK = comm.bcast(avgK, root=0)

In [10]:
%%px

print("kinectic energy = "+str(avgK))

[stdout:0] kinectic energy = 0.68327665329
[stdout:1] kinectic energy = 0.68327665329
[stdout:2] kinectic energy = 0.68327665329
[stdout:3] kinectic energy = 0.68327665329
[stdout:4] kinectic energy = 0.68327665329
[stdout:5] kinectic energy = 0.68327665329
[stdout:6] kinectic energy = 0.68327665329
[stdout:7] kinectic energy = 0.68327665329


In [11]:
%%px

comm.Barrier(); t1=MPI.Wtime()
# Get wavenumber:
myEnergySpc=EnergySpectrum()
kx,ky,kz=myEnergySpc.FindWavenumber(nx,ny,nz,rank)
k2=np.zeros((nx,ny,nz_half+1), dtype='float32')
np.copyto(k2,kx*kx+ky*ky+kz*kz)
k2[0,0,0]=1e-6

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate all wavenumbers: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate all wavenumbers: 55.59 seconds


In [12]:
%%px 

fft = FFT3Dfield_new(nx,ny,nz,nproc,rank)

In [13]:
%%px

cvx = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')
cvy = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')
cvz = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')

In [14]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cvx[:,:,:] = fft.forward3Dfft(vx,nx,ny,nz,nproc,rank)
cvy[:,:,:] = fft.forward3Dfft(vy,nx,ny,nz,nproc,rank)
cvz[:,:,:] = fft.forward3Dfft(vz,nx,ny,nz,nproc,rank)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate 3D spatial FFT: 364.80 seconds


In [15]:
%%px
comm.Barrier(); t1=MPI.Wtime()

cSxy=np.complex64(0.0+1.0j)*(kx*cvy+ky*cvx)
cSxz=np.complex64(0.0+1.0j)*(kx*cvz+kz*cvx)
cSyz=np.complex64(0.0+1.0j)*(ky*cvz+kz*cvy)

cSxx=np.complex64(0.0+1.0j)*(kx*cvx+kx*cvx)
cSyy=np.complex64(0.0+1.0j)*(ky*cvy+ky*cvy)
cSzz=np.complex64(0.0+1.0j)*(kz*cvz+kz*cvz)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate strain-rate in k space: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate strain-rate in k space: 387.99 seconds


In [16]:
%%px

cSxy[kz==Nz/2] = 0.
cSxz[kz==Nz/2] = 0.
cSyz[kz==Nz/2] = 0.

cSxx[kz==Nz/2] = 0.
cSyy[kz==Nz/2] = 0.
cSzz[kz==Nz/2] = 0.

In [17]:
%%px

Sxy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
Sxz = ft.zeros_aligned((nx,ny,nz), dtype='float32')
Syz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

Sxx = ft.zeros_aligned((nx,ny,nz), dtype='float32')
Syy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
Szz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

In [18]:
%%px
comm.Barrier(); t1=MPI.Wtime()
# Get velocity field in Fourier space:

Sxy[:,:,:]=fft.backward3Dfft(cSxy,nx,ny,nz,nproc,rank)
Sxz[:,:,:]=fft.backward3Dfft(cSxz,nx,ny,nz,nproc,rank)
Syz[:,:,:]=fft.backward3Dfft(cSyz,nx,ny,nz,nproc,rank)

Sxx[:,:,:]=fft.backward3Dfft(cSxx,nx,ny,nz,nproc,rank)
Syy[:,:,:]=fft.backward3Dfft(cSyy,nx,ny,nz,nproc,rank)
Szz[:,:,:]=fft.backward3Dfft(cSzz,nx,ny,nz,nproc,rank)

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate 3D spatial inverse FFT: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate 3D spatial inverse FFT: 514.06 seconds


In [19]:
%%px
comm.Barrier(); t1=MPI.Wtime()

S2 = Sxx**2 + Syy**2 + Szz**2 + 2.*Sxy**2 + 2.*Sxz**2 + 2*Syz**2

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Calculate dissipation rate: {0:.2f} seconds\n'.format(t2-t1))

[stdout:0] Calculate dissipation rate: 534.72 seconds


In [21]:
%%px

print("hello")

[stdout:0] hello
[stdout:1] hello
[stdout:2] hello
[stdout:3] hello
[stdout:4] hello
[stdout:5] hello
[stdout:6] hello
[stdout:7] hello


In [22]:
%%px

#slabS2 = np.sum(S2)
#avgS2=np.zeros(1,dtype='float32')

#comm.Reduce([slabS2,MPI.REAL],[avgS2,MPI.REAL],op=MPI.SUM)
#avgS2 = avgS2[0]*(1024**(-3))
#avgS2 = comm.bcast(avgS2, root=0)
avgS2loc = np.average(S2)
avgS2glob = np.zeros(1,dtype='float32')

In [23]:
%%px

comm.Reduce([avgS2loc,MPI.REAL],[avgS2glob,MPI.REAL],op=MPI.SUM)

In [24]:
%%px

avgS2glob = avgS2glob[0]/nproc
avgS2glob = comm.bcast(avgS2glob,root=0)

In [25]:
%%px

print("avgS2 = "+str(avgS2glob/4))

[stdout:0] avgS2 = 248.70942688
[stdout:1] avgS2 = 248.70942688
[stdout:2] avgS2 = 248.70942688
[stdout:3] avgS2 = 248.70942688
[stdout:4] avgS2 = 248.70942688
[stdout:5] avgS2 = 248.70942688
[stdout:6] avgS2 = 248.70942688
[stdout:7] avgS2 = 248.70942688


In [28]:
%%px

epsilon = 2*nu*(avgS2glob/4)
print("epsilon = "+str(2*nu*(avgS2glob/4)))

[stdout:0] epsilon = 0.0920224879456
[stdout:1] epsilon = 0.0920224879456
[stdout:2] epsilon = 0.0920224879456
[stdout:3] epsilon = 0.0920224879456
[stdout:4] epsilon = 0.0920224879456
[stdout:5] epsilon = 0.0920224879456
[stdout:6] epsilon = 0.0920224879456
[stdout:7] epsilon = 0.0920224879456


In [29]:
%%px
uprime = np.sqrt((2./3.)*avgK)
lamb = np.sqrt(15.*nu*(uprime**2/epsilon))
ReLamb = (uprime*lamb)/nu
if rank==0:
    print("uprime = "+str(uprime))
    print("lamb = "+str(lamb))
    print("ReLamb = "+str(ReLamb))

[stdout:0] 
uprime = 0.674920564852
lamb = 0.117202579368
ReLamb = 427.580708482


In [31]:
%%px

if rank==0:
    lamb = np.sqrt((2.*15.*nu)*(avgK/epsilon))
    eref = 0.103
    lamberef = np.sqrt((2.*15.*nu)*(avgK/eref))
    print("epsilon = "+str(2*nu*(avgS2glob/4)))
    print("kinectic energy = "+str(avgK))
    print("0) Taylor micro-scale ="+str(lamb))
    print("ReLamb = "+str(ReLamb))

[stdout:0] 
epsilon = 0.0920224879456
kinectic energy = 0.68327665329
0) Taylor micro-scale =0.203000822244
ReLamb = 427.580708482


In [36]:
%%px

if rank==0:
    ReKin = 0.6820399
    ReRef = 426.8378
    epref = (15./nu)*(((2*ReKin)/(3.*ReRef))**2)
    print(epref)

[stdout:4] 0.0920091102583509


At time t=1.024:
epsilon = 0.116420102539

kinectic energy = 0.692348361015

Taylor micro-scale =0.181674861195

ReLamb = 385.193709345

Reference:

epsilon = 0.09167584221268965

kinectic energy = 0.6915988

ReLamb = 433.6060

My dissipation rate is higher by a factor of 1.269910368 at timestep t=1.024

At time t=0.0: epsilon = 0.116845543823

kinectic energy = 0.682719171047

Taylor micro-scale =0.180078333764

ReLamb = 379.144296604

Reference:

epsilon = 0.0920091102583509

kinectic energy = 0.6820399

ReLamb = 426.8378


My dissipation rate is higher by a factor of 1.269934504 at timestep t=0.0

######################################################

Using Kun Yang's download

At time t=0.0: epsilon = 0.0920224879456

kinectic energy = 0.68327665329

Taylor micro-scale = 0.203000822244

ReLamb = 427.580708482

Reference:

epsilon = 0.0920091102583509

kinectic energy = 0.6820399

ReLamb = 426.8378

My dissipation rate is higher by a factor of 1.269934504 at timestep t=0.0